In [0]:
## get list of subject directories
proj_dir = '/home/jefan/sketchloop02/'
roi_dir = '/home/jgunn/neurosketch/surferROIs/'
contents_dir = os.listdir(roi_dir)

sub_dirs = []
for i in contents_dir:
    try:
        if i.split('_')[1]=='neurosketch':
            sub_dirs.append(i)
    except:
        pass

sub_dirs = sorted(sub_dirs)

# issue with 1207161
sub_dirs = [s for s in sub_dirs if s != '1207161_neurosketch']
sub_dirs = [s for s in sub_dirs if s != '1201161_neurosketch']

sub_dirs =  ['0110171_neurosketch', '0110172_neurosketch', '0111171_neurosketch', '0112171_neurosketch', '0112172_neurosketch', '0112173_neurosketch', '0113171_neurosketch', '0115174_neurosketch',
                 '0117171_neurosketch', '0118171_neurosketch', '0118172_neurosketch', '0119171_neurosketch', '0119172_neurosketch', '0119173_neurosketch', '0119174_neurosketch', '0120171_neurosketch',
                 '0120172_neurosketch', '0120173_neurosketch', '0123171_neurosketch', '0123173_neurosketch', '0124171_neurosketch', '0125171_neurosketch', '0125172_neurosketch', '1121161_neurosketch',
                 '1130161_neurosketch', '1202161_neurosketch', '1203161_neurosketch', '1206161_neurosketch', '1206162_neurosketch', '1206163_neurosketch', '1207162_neurosketch',
                 '0123171_neurosketch', '0123173_neurosketch', '0124171_neurosketch', '0125171_neurosketch', '0125172_neurosketch']

print(sub_dirs)
print(str(len(sub_dirs)) + ' subjects')

# Setup

In [0]:
import warnings
warnings.filterwarnings("ignore")

from __future__ import division

import numpy as np
import os
from glob import glob

from PIL import Image
from copy import deepcopy

from sklearn import linear_model, datasets, neighbors
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn import svm

%matplotlib inline
from scipy.misc import imread, imresize
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import seaborn as sns
sns.set_context('poster')
colors = sns.color_palette("cubehelix", 5)

import pandas as pd

import scipy.stats as stats
from scipy.stats import norm
import sklearn

from importlib import reload

### define paths to data

In [0]:
## root paths
curr_dir = os.getcwd()
proj_dir = 'C:\\Users\\jgunn\\Google Drive\\neurosketch'
data_dir = 'D:\\data'
results_dir = os.path.join(proj_dir, 'csv')
nb_name = 'clfsearchlight'

## add helpers to python path
import sys
if os.path.join(proj_dir, 'python') not in sys.path:
    sys.path.append(os.path.join(proj_dir, 'python'))

## module definitions
import analysis_helpers as h
reload(h)
h.data_dir = data_dir
h.path_to_recog = os.path.join(data_dir, 'features/recog')
h.path_to_draw = os.path.join(data_dir, 'features/drawing')
h.roi_list_draw = np.array(['V1Draw', 'V2Draw', 'LOCDraw', 'InsulaDraw', 'postCentralDraw',
                            'preCentralDraw', 'ParietalDraw', 'FrontalDraw', 'smgDraw'])
h.roi_list_recog = np.array(['V1', 'V2', 'LOC', 'fusiform','parahippo','IT','ento','PRC','hipp','mOFC'])
roi_list_recog = h.roi_list_recog

### get file list

In [0]:
## get raw file list for recognition runs
path_to_recog = h.path_to_recog

RECOG_METAS = sorted([i for i in os.listdir(path_to_recog) if (i.split('.')[-1]=='csv') & (i.split('_')[2][-4:] != 'Draw')])
RECOG_FEATS = sorted([i for i in os.listdir(path_to_recog) if (i.split('.')[-1]=='npy') & (i.split('_')[1][-4:] != 'Draw')])
RECOG_SUBS = np.array([i.split('_')[0] for i in RECOG_FEATS])

recog_sub_list = np.unique(RECOG_SUBS)

def preprocess_recog(RECOG_METAS, RECOG_FEATS):
    M = [i for i in RECOG_METAS if len(i.split('.')[0].split('_'))==4]
    F = [i for i in RECOG_FEATS if len(i.split('.')[0].split('_'))==4]
    return M,F

RECOG_METAS, RECOG_FEATS = preprocess_recog(RECOG_METAS, RECOG_FEATS)

In [0]:
## get raw file list for drawing runs
path_to_draw = h.path_to_draw

DRAW_METAS = sorted([i for i in os.listdir(path_to_draw) if (i.split('.')[-1]=='csv') & (i.split('_')[2][-4:] == 'Draw')])
DRAW_FEATS = sorted([i for i in os.listdir(path_to_draw) if (i.split('.')[-1]=='npy') & (i.split('_')[1][-4:] == 'Draw')])
DRAW_SUBS = np.array([i.split('_')[0] for i in DRAW_FEATS])
draw_sub_list = np.unique(DRAW_SUBS)

In [0]:
## get subject ID's that have complete datasets from all phases of experiment
sub_list = np.intersect1d(recog_sub_list,draw_sub_list)
#print('Number of subs: {}'.format(len(sub_list)))

In [0]:
## filter file list so only contains the sessions that have full datasets
def extract_good_sessions(DRAW_METAS,DRAW_FEATS,RECOG_METAS,RECOG_FEATS):
    _DRAW_METAS = [i for i in DRAW_METAS if i.split('_')[1] in sub_list]
    _DRAW_FEATS = [i for i in DRAW_FEATS if i.split('_')[0] in sub_list]
    _RECOG_METAS = [i for i in RECOG_METAS if i.split('_')[1] in sub_list]
    _RECOG_FEATS = [i for i in RECOG_FEATS if i.split('_')[0] in sub_list]
    return _DRAW_METAS, _DRAW_FEATS, _RECOG_METAS, _RECOG_FEATS

DRAW_METAS,DRAW_FEATS,RECOG_METAS,RECOG_FEATS =  \
extract_good_sessions(DRAW_METAS,DRAW_FEATS,RECOG_METAS,RECOG_FEATS)

RECOG_SUBS = np.array([i.split('_')[0]+'_neurosketch' for i in RECOG_FEATS])
RECOG_ROIS = np.array([i.split('_')[1] for i in RECOG_FEATS])

DRAW_SUBS = np.array([i.split('_')[0]+'_neurosketch' for i in DRAW_FEATS])
DRAW_ROIS = np.array([i.split('_')[1] for i in DRAW_FEATS])

In [0]:
sub_list

['0110171', '0110172', '0111171', '0112171', '0112172', '0112173',
       '0113171', '0115174', '0117171', '0118171', '0118172', '0119171',
       '0119172', '0119173', '0119174', '0120171', '0120172', '0120173',
       '0123171', '0123173', '0124171', '0125171', '0125172', '1121161',
       '1130161', '1202161', '1203161', '1206161', '1206162', '1206163',
       '1207162']

array(['0110171', '0110172', '0111171', '0112171', '0112172', '0112173',
       '0113171', '0115174', '0117171', '0118171', '0118172', '0119171',
       '0119172', '0119173', '0119174', '0120171', '0120172', '0120173',
       '0123171', '0123173', '0124171', '0125171', '0125172', '1121161',
       '1130161', '1202161', '1203161', '1206161', '1206162', '1206163',
       '1207162'], dtype='<U7')

# Collect Feature Set
We want 160 x num_voxels feature array saved as .npy for each subject, focused on occitemp.

In [0]:
sub_list = ['0110171', '0110172', '0111171', '0112171', '0112172', '0112173',
            '0113171', '0115174', '0117171', '0118171', '0118172', '0119171',
            '0119172', '0119173', '0119174', '0120171', '0120172', '0120173',
            '0123171', '0123173', '0124171', '0125171', '0125172', '1121161',
            '1130161', '1202161', '1203161', '1206161', '1206162', '1206163',
            '1207162']

for subject in sub_list:
    print(subject)

    for phase in ['12', '34', '56']: # WE ONLY NEED THE FIRST PHASE AT THE MOMENT. CHANGE THIS IF YOU WANT EVERY PHASE.
        print(phase)

        subj = [subject] * 160
        label = []
        run_num = [1]*80 + [2]*80
        TR_num = []
        features = []

        for r in phase:
            run = int(r)

            # load subject's time series for this run
            # /jukebox/ntb/users/jwammes/sketchloop/subjects/#######_neurosketch/analysis/firstlevel/parameter/#######_neurosketch_3mm_drawing_run_1_filtfuncHIRES.nii.gz
            timeseries = image.load_img('/jukebox/ntb/users/jwammes/sketchloop/subjects/' + subject[:7] + 
                                        '_neurosketch/analysis/firstlevel/parameter/' + str(run) + '_hires.nii.gz')
            timeseries = timeseries.get_data().transpose((3, 0, 1, 2))

            # use information in regressor/run_x folder to make hasImage vector
            # associated TR is just the hasImage index, converted to a float if needed
            hasImage = [0]*240
            for cope in ['bed', 'bench', 'chair', 'table']:
                with open('/home/jgunn/neurosketch/timepoints/' + subject[:7] + '_' + str(run) + '_' + cope + '.txt') as f:
                    times = [line.split(' ')[0] for line in f.read().split('\n')[:-1]]
                    for t in times:
                        tr = float(t)/1.5
                        if cope == 'bed':
                            hasImage[int(tr)] = 1
                        elif cope == 'bench':
                            hasImage[int(tr)] = 2
                        elif cope == 'chair':
                            hasImage[int(tr)] = 3
                        elif cope == 'table':
                            hasImage[int(tr)] = 4

            # wherever hasImage, we want the features
            count = 0
            for i, has in enumerate(hasImage): # 80 times
                if has != 0:
                    count += 1
                    features.append(timeseries[i+3])

            # gotta track label and TR_num
            for index, value in enumerate(hasImage):
                if value != 0:
                    TR_num.append(float(index))
                    if value == 1:
                        label.append('bed')
                    elif value == 2:
                        label.append('bench')
                    elif value == 3:
                        label.append('chair')
                    elif value == 4:
                        label.append('table')

        features = np.stack(features,axis=1) # will have to take transpose later beacause of this
        #np.save(file='/home/jgunn/neurosketch/recmatrices/' + subject[:7] + '_' + roi + '_' + phase + '_featurematrix.npy', arr=features)

        x = pd.DataFrame([subj, label, run_num, TR_num]) # where each of those variables are lists of the same length
        x = x.transpose()
        x.columns = ['subj','label','run_num', 'TR_num']
        #x.to_csv('/home/jgunn/neurosketch/recmatrices/metadata_' + subject[:7] + '_' + roi  + '_' + phase + '.csv') # in a notebook, you can look at x by evaluating it by itself in its own cell.